In [8]:
import database_sql as db
import sqlite3
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
import os

In [2]:
dict_pdfs = {
    "nuevos": ["../01_data/2022202200133100001545.pdf", "../01_data/U0924996.pdf"]
}

In [ ]:
pdfs = [("../01_data/BBDD_Normativa_UPV/2022202200133100001545.pdf", 12121), ("../01_data/U0924996.pdf", 232)]

for pdf_info in pdfs:
        # First element is the path, second is the name
        path = pdf_info[0]
        id = pdf_info[1]

        # Load
        loader = PyPDFLoader(path)
        documentos = loader.load()

        # Split the documents into chunks of text
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
        documentos_divididos = text_splitter.split_documents(documentos)

        for i, doc in enumerate(documentos_divididos):
            # Get the page number
            page = doc.metadata.get('page_label', f"pagina_{i}")

            # Get the document ID
            doc_id = f"{id}_{i}"
            print(page, doc_id)

[Document(metadata={'producer': 'iText® 5.1.2 ©2000-2011 1T3XT BVBA', 'creator': 'Microsoft® Word 2010', 'creationdate': '2022-05-06T12:29:33+02:00', 'author': 'adm', 'moddate': '2025-03-08T12:45:26+01:00', 'source': '../01_data/BBDD_Normativa_UPV/2022202200133100001545.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='Núm. 074/2022 06/05/2022 2 \n \n \nEditor: Secretaria General / UPV ∙ D.L.: V‐5092‐2006 ∙ ISSN: 1887‐2298 ∙ www.upv.es/secgen \nANEXO \n \nMODIFICACIÓN CALENDARIO  ACADÉMICO UNIVERSITARIO DEL CURSO 2022/2023 \n \nAprobado por el Consejo de Gobierno en sesión de 28 de abril de 2022 \n \nEl calendario académico del curso 2022-2023 fue aprobado por Consejo de Gobierno de 23 de \ndiciembre de 2021. \n \nAdvertido error material en la relación de festivos de ámbito nacional, en la que se ha omitido \nel festivo correspondiente al 1 de mayo, día del Trabajo, procede la rectificación de la misma \npara su inclusión. \n \nPor otra parte, la Normativa de Preins

ValueError: File path ../01_data/U0924996.pdf is not a valid file or url

In [3]:
# Cargar el PDF de forma asíncrona
for file_path in dict_pdfs["nuevos"]:
    loader = PyPDFLoader(file_path)
    pages = []
    async for page in loader.alazy_load():
        pages.append(page)

In [4]:
page

Document(metadata={'producer': 'Antenna House PDF Output Library 6.6.1477 (Linux64)', 'creator': 'eBOE', 'creationdate': '2022-10-18T23:28:16+01:00', 'keywords': 'DECRETO 889/2022 de 18/10/2022;MINISTERIO DE UNIVERSIDADES;BOE-A-2022-17045;BOE 251 de 2022;17045;19/10/2022', 'moddate': '2022-10-18T23:39:43+02:00', 'trapped': '/False', 'subject': 'BOE-A-2022-17045', 'author': 'MINISTERIO DE UNIVERSIDADES', 'title': 'Disposición 17045 del BOE núm. 251 de 2022', 'source': '../01_data/U0924996.pdf', 'total_pages': 28, 'page': 27, 'page_label': '28'}, page_content='Orden ECI/3858/2007, de 27 de diciembre, por la que se establecen los requisitos \npara la verificación de los títulos universitarios oficiales que habiliten para el ejercicio de \nlas profesiones de Profesor de Educación Secundaria Obligatoria y Bachillerato, \nFormación Profesional y Enseñanzas de Idiomas.\nReal Decreto 775/2011, de 3 de junio, por el que se aprueba el Reglamento de la \nLey 34/2006, de 30 de octubre, sobre el ac

In [5]:
# Crear un text splitter que divida el texto en chunks de hasta 512 tokens
# Puedes ajustar el solapamiento (chunk_overlap) según lo requieras para preservar el contexto
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"

# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Función para contar tokens usando el tokenizador de Hugging Face
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text, add_special_tokens=False))

# Crear un divisor de texto basado en la cuenta de tokens
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,  # Número máximo de tokens por fragmento
    chunk_overlap=50,  # Superposición opcional entre fragmentos
    length_function=count_tokens,  # Usa el tokenizador de Hugging Face para contar tokens
)

In [6]:
# Procesar cada página y dividirla en fragmentos (chunks)
chunks = []
for page in pages:
    # Se divide el contenido de la página en fragmentos de hasta 512 tokens
    splitted_texts = text_splitter.split_text(page.page_content) # UNA LISTA DE STRINGS

    for chunk_text in splitted_texts:
        # Creamos un objeto Document para cada fragmento, manteniendo la metadata (por ejemplo, número de página)
        chunks.append(Document(page_content=chunk_text, metadata={"page": page.metadata.get("page"), "file": page.metadata.get("source").split("/")[-1]}))

In [7]:
chunks[20]

Document(metadata={'page': 7, 'file': 'U0924996.pdf'}, page_content='n.º 1024/2012 relativo a la cooperación administrativa a través del Sistema de \nInformación del Mercado Interior (Reglamento IMI).\nArtículo 4.\u2003Exclusiones.\n1.\u2003No podrá concederse la homologación, ni la declaración de equivalencia de \ntítulos extranjeros, a:\na)\u2003Títulos y diplomas propios, especialmente de formación permanente, que \nimpartan las universidades.\nb)\u2003Títulos españoles cuyos planes de estudios se hayan extinguido o que aún no \nestén implantados en al menos una universidad española.\nc)\u2003Niveles académicos distintos de Grado y Máster Universitario. En el caso del \nnivel académico de Doctorado, se aplicará el procedimiento establecido en la disposición \nadicional segunda.\n2.\u2003No serán objeto de homologación, ni de declaración de equivalencia, los \nsiguientes títulos extranjeros o estudios expedidos o realizados en el extranjero:\na)\u2003En los supuestos de homologación 

In [9]:
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

vector_store = Chroma.from_documents(chunks, embeddings, persist_directory="chroma_db")
vector_store.persist()

# Cargar base de datos Chroma
vector_store = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

/var/folders/1w/2mpv0jw11qj_v4tpnrdqv7t40000gn/T/ipykernel_3627/3683623978.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()
/var/folders/1w/2mpv0jw11qj_v4tpnrdqv7t40000gn/T/ipykernel_3627/3683623978.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory="chroma_db", embedding_function=embeddings)


In [12]:
# Realizar una búsqueda de similitud para obtener los chunks más relevantes
query = "¿Qué condiciones y limitaciones establece el Real Decreto 1325/2003 para que las personas beneficiarias del régimen de protección temporal sustituyan la documentación exigida por una declaración responsable en el inicio del procedimiento?"
retrieved_docs_with_scores = vector_store.similarity_search_with_score(query, k=5)

In [ ]:
contexto = retrieved_docs_with_scores[0][0].page_content
score = retrieved_docs_with_scores[0][1]

0.74338776

In [21]:
prompt = f"Resuelve este consulta en Español usando la información explicada en el fragmento Contexto: {contexto} Pregunta: {query}"

In [13]:
import ollama

response = ollama.chat(model="deepseek-r1:14b", messages=[{"role": "user", "content": prompt}])
print(response["message"]["content"])

<think>
Ok, así que tengo que resolver una pregunta sobre un real decreto español, específicamente el Real Decreto 1325/2003. La pregunta es: ¿Qué condiciones y limitaciones establece el Real Decreto 1325/2003 para que las personas beneficiarias del régimen de protección temporal sustituyan la documentación exigida por una declaración responsable en el inicio del procedimiento?

Primero, necesito entender bien lo que se me está preguntando. La persona quiere saber cuáles son las condiciones y limitaciones que establece dicho real decreto para que ciertas personas puedan usar una declaración responsable en lugar de la documentación requerida al iniciar un procedimiento.

Del contexto proporcionado, veo que el Real Decreto habla de las "Personas beneficiarias del régimen de protección temporal". En particular, menciona los artículos 2 y 11 del Reglamento sobre régimen de protección temporal en caso de afluencia masiva de personas desplazadas. También dice que estas personas pueden sustit